# **EDA & Feature Engneering**

## 📥 Load Data from AWS S3 Bucket

⚠️ No accesdetails needed for this Project

In [2]:
import pandas as pd


def load_flights_bronze():
    url = "https://lakehouse-masteryproject-2025.s3.eu-north-1.amazonaws.com/bronze/public_flights_export_2025-03-31_134734.csv"
    return pd.read_csv(url)

def load_hotels_bronze():
    url = "https://lakehouse-masteryproject-2025.s3.eu-north-1.amazonaws.com/bronze/public_hotels_export_2025-03-31_171805.csv"
    return pd.read_csv(url)

def load_sessions_bronze():
    url = "https://lakehouse-masteryproject-2025.s3.eu-north-1.amazonaws.com/bronze/public_sessions_export_2025-03-31_221253.csv"
    return pd.read_csv(url)

def load_users_bronze():
    url = "https://lakehouse-masteryproject-2025.s3.eu-north-1.amazonaws.com/bronze/public_users_export_2025-04-01_101058.csv"
    return pd.read_csv(url)

In [3]:
df_flights_bronze = load_flights_bronze()
df_hotels_bronze = load_hotels_bronze()
df_sessions_bronze = load_sessions_bronze()
df_users_bronze = load_users_bronze()

print("✅ Flight, hotel, session and user data loaded successfully.")

✅ Flight, hotel, session and user data loaded successfully.


## 🔍Reducing Columns

The next step is to reduce the columns to only those needed to validate the hypotheses behind the defined perks. All other columns are excluded unless they are required to calculate new features (e.g., `nights` for trip duration) or to support the interpretation of results.

🧠 Personalization Scope (Project Context)

In this project, **personalization** is based exclusively on **individual user behavior** observed within the TravelTide platform. This includes patterns such as discount usage, booking behavior, and cancellation frequency.

Demographic or group-level attributes (e.g. family status, number of travelers, or room count) are not considered. The goal is to assign each user the perk most relevant to **how they interact** with the platform — not who they are or who they travel with.



### 👤 Reducing Columns – `users` Table



For the `users` table, we keep only the columns required to support the behavioral logic behind the defined perks. Since personalization in this project is based purely on **individual interaction behavior**, we exclude all demographic and group-related attributes.


| Column         | Description                                                           | Keep? |
|----------------|------------------------------------------------------------------------|:-----:|
| `user_id`      | Required to join with sessions and flights                            | ✅    |
| `birthdate`    | May support age-related behavioral segmentation if needed             | ✅    |
| `has_children` | Not used for any defined perk — group/family-related                  | ❌    |
| `gender`       | Not tied to any perk                                                  | ❌    |
| `married`      | Not tied to any perk                                                  | ❌    |
| `home_country` | Not required for perk logic                                           | ❌    |
| `home_city`    | Too granular for this scope                                           | ❌    |
| `home_airport` | Not relevant for current perks                                        | ❌    |
| `sign_up_date` | Not used in perk logic                                                | ❌    |


In [14]:

df_users_bronze = df_users_bronze[["user_id", "birthdate"]]

df_users_bronze.head()


,user_id,birthdate
0,0,1990-01-22
1,1,2000-11-08
2,2,1992-09-21
3,3,1996-11-27
4,4,1978-01-05


### 🧾 Reducing Columns – `sessions` Table


The `sessions` table is the most important source for behavioral signals in this project. We retain only the columns directly required to support the five defined perks or essential to filtering and joining.

| Column                   | Description                                                                 | Keep? |
|--------------------------|-----------------------------------------------------------------------------|:-----:|
| `session_id`             | Needed to count sessions per user (7+ sessions filter)                      | ✅    |
| `user_id`                | Required to join with users                                                 | ✅    |
| `trip_id`                | Required to connect to flights and hotels                                   | ✅    |
| `session_start`          | Used to filter sessions after Jan 4, 2023                                   | ✅    |
| `flight_discount`        | Supports the "exclusive discounts" perk                                     | ✅    |
| `hotel_discount`         | Supports the "exclusive discounts" perk                                     | ✅    |
| `flight_discount_amount` | Captures price sensitivity for flight discounts                             | ✅    |
| `hotel_discount_amount`  | Captures price sensitivity for hotel discounts                              | ✅    |
| `flight_booked`          | Required to understand booking behavior                                     | ✅    |
| `hotel_booked`           | Required to validate hotel+flight pairing                                   | ✅    |
| `page_clicks`            | Indicates user engagement — helps understand interaction effort             | ✅    |
| `cancellation`           | Directly supports the "no cancellation fees" perk                           | ✅    |
| `session_end`            | Not required for any of the defined perks                                   | ❌    |


In [15]:
df_sessions_bronze = df_sessions_bronze[
    [
        "session_id", "user_id", "trip_id", "session_start",
        "flight_discount", "hotel_discount",
        "flight_discount_amount", "hotel_discount_amount",
        "flight_booked", "hotel_booked",
        "page_clicks", "cancellation"
    ]
]
df_sessions_bronze.head()

,session_id,user_id,trip_id,session_start,flight_discount,hotel_discount,flight_discount_amount,hotel_discount_amount,flight_booked,hotel_booked,page_clicks,cancellation
0,841803-95c93659053948049c73106ffdcf4df6,841803,841803-31d33d39b08f477bb8d9dd3b0dcbf1d3,2023-05-17 19:40:00,False,False,NaN,NaN,True,True,30,False
1,841804-bf1bfe452c4d4372b04d9612134006e7,841804,NaN,2023-05-17 22:40:00,False,False,NaN,NaN,False,False,9,False
2,841805-70d695ee6540481bae7750e368c10443,841805,841805-506d17340ffa41659736fd258138a9b5,2023-05-17 21:10:00,False,False,NaN,NaN,True,True,24,False
3,841806-a03ac981d1c847979765669ca871a0cb,841806,NaN,2023-05-17 21:49:00,True,True,0.2,0.2,False,False,42,False
4,841807-4febb023cd414a6196fc026af1928bfb,841807,NaN,2023-05-17 20:28:00,False,False,NaN,NaN,False,False,4,False


### ✈️ Reducing Columns – `flights` Table



We retain only the columns that are directly required to support perk validation. Columns related to destinations, time, or route details are excluded, as none of the five defined perks depend on them.

| Column                   | Description                                                                                  |
|--------------------------|----------------------------------------------------------------------------------------------|
| `trip_id`                | ✅ Required to join with sessions                                                             |
| `checked_bags`           | ✅ Directly supports the "free checked bag" perk                                              |
| `base_fare_usd`          | ✅ Supports the "exclusive discounts" perk by capturing price sensitivity                     |
| `trip_airline`           | ❌ Not needed for current perks                                                               |
| `origin_airport`         | ❌ Not required for any defined perk                                                          |
| `destination`            | ❌ Not required for any defined perk                                                          |
| `destination_airport`    | ❌ Redundant and not perk-related                                                             |
| `seats`                  | ❌ Not needed for defined perks                                                               |
| `return_flight_booked`   | ❌ Not used for any current perk                                                              |
| `departure_time`         | ❌ Not used for any current perk                                                              |
| `return_time`            | ❌ Not used for any current perk                                                              |
| `destination_airport_lat`| ❌ Mapping only — not used in perks                                                           |
| `destination_airport_lon`| ❌ Mapping only — not used in perks                                                           |


In [16]:
df_flights_bronze = df_flights_bronze[
    ["trip_id", "checked_bags", "base_fare_usd"]
]

df_flights_bronze.head()


,trip_id,checked_bags,base_fare_usd
0,693092-7b8767746abd45219ed468d58da9f702,0,479.31
1,693124-1032149a9e01427da914b17ce6ef1926,0,575.78
2,693149-0fdb31c7dc114a4d8a318f0feada365c,0,336.72
3,693169-d120c3721d0b4bcca278639c981e4322,0,295.37
4,693181-f4a2363db0ee452f8ac8c0fe3d4ef0ad,1,694.26


### 🏨 Reducing Columns – `hotels` Table

In the `hotels` table, we only keep columns that are directly needed to validate the defined perks. All other fields, such as room count, hotel brand, or check-in/out times, are excluded since they are not required for any of the five perks.


| Column               | Description                                                                                   |
|----------------------|-----------------------------------------------------------------------------------------------|
| `trip_id`            | ✅ Required to join hotels with sessions and flights                                          |
| `nights`             | ✅ Required to validate the "1 night free hotel with flight" perk                             |
| `hotel_per_room_usd` | ✅ Supports the "exclusive discounts" perk by capturing price sensitivity                      |
| `hotel_name`         | ❌ Not needed for any defined perk                                                             |
| `rooms`              | ❌ Not used for any current perk                                                               |
| `check_in_time`      | ❌ Not required for any of the defined perks                                                   |
| `check_out_time`     | ❌ Not required — duration is already captured by `nights`                                     |


In [ ]:
df_hotels_bronze = df_hotels_bronze.drop(columns=["nights"])

Instead of relying on the nights column from the dataset, we calculate it ourselves based on actual hote logic: check-in time, check-out time, and a grace period. A late checkout may result in an additional night being counted. This behavior is more realistic.

In [ ]:
from datetime import datetime, timedelta
import pandas as pd

# Hotel rules
CHECKIN_TIME = datetime.strptime("15:00", "%H:%M").time()
CHECKOUT_TIME = datetime.strptime("11:00", "%H:%M").time()
GRACE_PERIOD = timedelta(hours=1)  # 1-hour grace period

# Function to calculate nights based on check-in/out times
def calculate_nights_realistic(row):
    check_in = row["check_in_time"]
    check_out = row["check_out_time"]

    if pd.isna(check_in) or pd.isna(check_out):
        return None

    check_in_date = check_in.date()
    check_out_date = check_out.date()

    # Base night count = number of calendar days
    nights = (check_out_date - check_in_date).days

    # Add 1 night if checkout was late (after grace period)
    checkout_deadline = datetime.combine(check_out_date, CHECKOUT_TIME) + GRACE_PERIOD
    if check_out > checkout_deadline:
        nights += 1

    return nights

# Apply to dataset
df_hotels_bronze["nights"] = df_hotels_bronze.apply(calculate_nights_realistic, axis=1)


In [18]:
df_hotels_bronze = df_hotels_bronze[["trip_id", "nights", "hotel_per_room_usd"]]

df_hotels_bronze.head()

,trip_id,nights,hotel_per_room_usd
0,234666-99574aa53f30402ba04bfe2f3337c566,2,226
1,234707-4c0cfdb5ff4a4cd8bda5feed22f790d7,1,114
2,234745-a8f944dc121c4003b16c108c9c590965,3,322
3,234905-c1e14087ef1f4a7a8413a12ef4208135,2,228
4,235101-cd0e752033db4f488fa75b5eb8feb2c8,1,163


## 🔄 Parse and Standardize Data Types

Before we can assign perks or create user-level aggregations, we need to ensure that all four tables — `users`, `sessions`, `flights`, and `hotels` — are clean and consistent. This includes standardizing data types, handling missing values, and ensuring that binary and datetime fields are correctly formatted.

Each table requires slightly different steps, depending on the structure of the data it contains.

### 👤 Data Preparation – `users` Table

The `users` table contains only two columns needed for the defined perks: `user_id` and `birthdate`. Our main goal here is to ensure `birthdate` is in datetime format so we can calculate user age if needed.


Step 1 – Convert Columns to Correct Data Types


In [69]:
df_users_bronze["birthdate"] = pd.to_datetime(df_users_bronze["birthdate"], errors="coerce")

Step 2 – Handle Missing Values

In [71]:
# Check for nulls
df_users_bronze.isna().sum()

# Drop rows with missing user_id or birthdate
df_users_bronze = df_users_bronze.dropna(subset=["user_id", "birthdate"])


In [72]:
df_users_bronze.head()

,user_id,birthdate
0,0,1990-01-22
1,1,2000-11-08
2,2,1992-09-21
3,3,1996-11-27
4,4,1978-01-05


### 🧾 Data Preparation – `sessions` Table

We start by preparing the `sessions` table. The main focus is on:

- Ensuring correct data types (especially for datetime and boolean fields)
- Handling missing values
- Verifying binary logic for downstream scoring and aggregation

Step 1 – Convert Columns to Correct Data Types

In [21]:
df_sessions_bronze["session_start"] = pd.to_datetime(df_sessions_bronze["session_start"])
df_sessions_bronze["flight_discount"] = df_sessions_bronze["flight_discount"].astype(bool)
df_sessions_bronze["hotel_discount"] = df_sessions_bronze["hotel_discount"].astype(bool)
df_sessions_bronze["flight_booked"] = df_sessions_bronze["flight_booked"].astype(bool)
df_sessions_bronze["hotel_booked"] = df_sessions_bronze["hotel_booked"].astype(bool)
df_sessions_bronze["cancellation"] = df_sessions_bronze["cancellation"].astype(bool)

Step 2 – Handle Missing Values

In [22]:
# Check for nulls
df_sessions_bronze.isna().sum()

# Drop sessions without user_id or trip_id, which are critical
df_sessions_bronze = df_sessions_bronze.dropna(subset=["user_id", "trip_id"])

# Fill missing discount amounts with 0
df_sessions_bronze["flight_discount_amount"] = df_sessions_bronze["flight_discount_amount"].fillna(0)
df_sessions_bronze["hotel_discount_amount"] = df_sessions_bronze["hotel_discount_amount"].fillna(0)


Step 3 – Validate Binary Columns

In [23]:
# Confirm binary fields contain only True/False
binary_fields = ["flight_discount", "hotel_discount", "flight_booked", "hotel_booked", "cancellation"]
for col in binary_fields:
    print(col, df_sessions_bronze[col].unique())


flight_discount [False  True]
hotel_discount [False  True]
flight_booked [ True False]
hotel_booked [ True False]
cancellation [False  True]


### ✈️ Data Preparation – `flights` Table

In the `flights` table, we prepare the data by ensuring correct data types and handling missing values in price and baggage-related columns. Since only three columns are needed to validate the defined perks, we focus cleanup on those specifically.


Step 1 – Convert Columns to Correct Data Types

In [27]:
df_flights_bronze["checked_bags"] = pd.to_numeric(df_flights_bronze["checked_bags"], errors="coerce")
df_flights_bronze["base_fare_usd"] = pd.to_numeric(df_flights_bronze["base_fare_usd"], errors="coerce")

Step 2 – Handle Missing Values

In [28]:
# Check for nulls
df_flights_bronze.isna().sum()

# Drop rows missing critical fields
df_flights_bronze = df_flights_bronze.dropna(subset=["trip_id", "checked_bags", "base_fare_usd"])

### 🏨 Data Preparation – `hotels` Table

In the `hotels` table, we focus on ensuring that time and price fields are correctly formatted and that missing values are handled. This is especially important for calculating `nights` from `check_in_time` and `check_out_time`.


Step 1 – Convert Columns to Correct Data Types

In [32]:
df_hotels_bronze["nights"] = pd.to_numeric(df_hotels_bronze["nights"], errors="coerce")
df_hotels_bronze["hotel_per_room_usd"] = pd.to_numeric(df_hotels_bronze["hotel_per_room_usd"], errors="coerce")

Step 2 – Handle Missing Values

In [35]:
# Check for nulls
df_hotels_bronze.isna().sum()

# Drop rows with missing trip_id or calculated nights
df_hotels_bronze = df_hotels_bronze.dropna(subset=["trip_id", "nights"])


## 💾 Save Silver Tables for Manual Upload to AWS S3 Bucket

In [75]:
import pandas as pd
import os

# Silver DataFrames (copies of bronze, after cleaning)
df_flights_silver = df_flights_bronze
df_hotels_silver = df_hotels_bronze
df_sessions_silver = df_sessions_bronze
df_users_silver = df_users_bronze

# Folder to save locally before manual S3 upload
silver_local_path = "silver_exports/"
os.makedirs(silver_local_path, exist_ok=True)

# Tables to save
tables = {
    "flights_silver": df_flights_silver,
    "users_silver": df_users_silver,
    "hotels_silver": df_hotels_silver,
    "sessions_silver": df_sessions_silver
}

# Save each table as CSV locally
for name, df in tables.items():
    try:
        path = os.path.join(silver_local_path, f"{name}.csv")
        df.to_csv(path, index=False)
        print(f"✅ Exported: {path}")
    except Exception as e:
        print(f"❌ Error in {name}: {e}")


✅ Exported: silver_exports/flights_silver.csv
✅ Exported: silver_exports/users_silver.csv
✅ Exported: silver_exports/hotels_silver.csv
✅ Exported: silver_exports/sessions_silver.csv


## 💾 Gold Level Preparation & Save Gold Tables for Manual Upload to AWS S3 Bucket

Before aggregating features and clustering, we filter the silver data to include only users who meet the cohort condition defined during EDA.

### 📎 Filter Cohort – Users with ≥ 7 Sessions Since Jan 4, 2023

In [76]:
import pandas as pd

# 1. Filter sessions since January 4, 2023
df_sessions_silver["session_start"] = pd.to_datetime(df_sessions_silver["session_start"])
recent_sessions = df_sessions_silver[df_sessions_silver["session_start"] >= "2023-01-04"]

# 2. Count sessions per user and select those with ≥ 7
session_counts = recent_sessions["user_id"].value_counts()
eligible_user_ids = session_counts[session_counts >= 7].index

# 3. Filter all silver tables to include only eligible users
df_sessions_silver = df_sessions_silver[df_sessions_silver["user_id"].isin(eligible_user_ids)]
df_users_silver = df_users_silver[df_users_silver["user_id"].isin(eligible_user_ids)]

# 4. Keep only trips linked to eligible users for hotel/flight data
valid_trip_ids = df_sessions_silver["trip_id"].dropna().unique()
df_flights_silver = df_flights_silver[df_flights_silver["trip_id"].isin(valid_trip_ids)]
df_hotels_silver = df_hotels_silver[df_hotels_silver["trip_id"].isin(valid_trip_ids)]

print(f"✅ Filtered to cohort: {len(eligible_user_ids)} users with ≥ 7 sessions since 2023-01-04.")

✅ Filtered to cohort: 79 users with ≥ 7 sessions since 2023-01-04.


🟡 Export Gold Layer – Final User-Cohort Data

In [77]:
import pandas as pd
import os

# Gold DataFrames (filtered for cohort and ready for clustering)
df_flights_gold = df_flights_silver
df_hotels_gold = df_hotels_silver
df_sessions_gold = df_sessions_silver
df_users_gold = df_users_silver

# Folder to save locally before manual S3 upload
gold_local_path = "gold_exports/"
os.makedirs(gold_local_path, exist_ok=True)

# Tables to save
tables = {
    "flights_gold": df_flights_gold,
    "users_gold": df_users_gold,
    "hotels_gold": df_hotels_gold,
    "sessions_gold": df_sessions_gold
}

# Save each table as CSV locally
for name, df in tables.items():
    try:
        path = os.path.join(gold_local_path, f"{name}.csv")
        df.to_csv(path, index=False)
        print(f"✅ Exported: {path}")
    except Exception as e:
        print(f"❌ Error in {name}: {e}")


✅ Exported: gold_exports/flights_gold.csv
✅ Exported: gold_exports/users_gold.csv
✅ Exported: gold_exports/hotels_gold.csv
✅ Exported: gold_exports/sessions_gold.csv


# **Customer Segmentation**

Now that the data is cleaned and all perk-relevant features have been engineered, we begin the segmentation phase.

The goal is to group customers based on their behavior and assign the most suitable perk to each group.


### ✅ Step Overview



1. **Aggregate metrics at user level**  
   - Calculate behavioral features (e.g., number of sessions, cancellation rate, discount usage)

2. **Standardize features for clustering**  
   - Normalize or scale numeric values (e.g., MinMaxScaler)

3. **Apply clustering algorithm**  
   - Use unsupervised learning (e.g., K-Means or DBSCAN) to form customer groups

4. **Explore and interpret the clusters**  
   - Analyze cluster composition to understand behavioral patterns

5. **Assign best-fitting perk to each group**  
   - Match perk logic to dominant behavior in each cluster

---

The outcome of this step will be:
- A set of behaviorally meaningful user segments
- Each segment matched to a perk based on its collective characteristics

### 📊 Step 1 – User-Level Aggregation

To segment customers meaningfully, we first create a single user-level DataFrame that aggregates all relevant behavioral metrics from the cleaned `silver` tables.

These features will serve as input for clustering.

✅ Features to Aggregate Per User

| Feature                     | Source Table        | Description                                                                 |
|-----------------------------|---------------------|-----------------------------------------------------------------------------|
| `total_sessions`           | sessions_silver     | Total number of sessions per user                                           |
| `cancellation_rate`        | sessions_silver     | Share of sessions marked as cancellations                                   |
| `discount_usage_rate`      | sessions_silver     | Share of sessions where any discount was applied                            |
| `total_nights`             | hotels_silver       | Total number of nights across all hotel bookings                            |
| `total_checked_bags`       | flights_silver      | Total number of checked bags across all flights                             |
| `total_base_fare`          | flights_silver      | Sum of base flight fares — indicates price sensitivity or booking class     |
